# IMPORTING REQUIRED LIBRARIES

In [1]:
from selenium import webdriver                          #For opening browser       
import pandas as pd                                     #For creating dataframe
from bs4 import BeautifulSoup                           #For scraping data
import time                                             #For waiting
import numpy as np                                      #For array
from tqdm import tqdm                                   #For progress bar

# OPENING REQUIRED SITE IN BROWSER

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://m.imdb.com/chart/starmeter/')

# GETTING HTML CODE OF OPENED WEBPAGE

In [4]:
html = driver.page_source

# CONVERTING EXTRACTED HTML CODE INTO SOUP TO SCRAP DATA

In [5]:
soup = BeautifulSoup(html,"html.parser")

# EXTRACTING NAMES OF CELEBRITIES AND LINKS TO THEIR INDIVIDUAL WEBPAGES

In [6]:
a_list = soup.find_all("a")
name_list = []
link_list = []
for i in range(43,343,3):
    name_list.append(a_list[i].get('aria-label'))
    link_list.append(a_list[i].get('href'))

# EXTRACTING THE FAMOUS WORK OF THESE CELEBRITIES

In [7]:
famous_list = []
a_list1 = soup.findAll('a',{'data-testid':'nlib-known-for-title'})
for i in a_list1:
    famous_list.append(i.text)

# FUNCTION TO EXTRACT DATE OF BIRTH OF THE CELEBRITIES

In [8]:
def get_dob(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        dob = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[0].text
    except Exception as e:
        dob = np.nan
    return dob

# FUNCTION TO EXTRACT HEIGHT OF THE CELEBRITIES

In [9]:
def get_height(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        height = sec.findAll('li',{'data-testid':'nm_pd_he'})[0].findAll('li')[0].text
    except Exception as e:
        height = np.nan
    return height

# FUNCTION TO EXTRACT BIRTHPLACE OF THE CELEBRITIES

In [10]:
def get_birthplace(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        bp = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[1].text
    except Exception as e:
        bp = np.nan
    return bp

# FUNCTION TO EXTRACT AWARD DATA OF THE CELEBRITIES

In [11]:
def get_awards(soup):
    try:
        div_list = soup.findAll('div', {'data-testid':'awards'})
        awards = div_list[0].findAll('li',{'class':'ipc-inline-list__item'})[0].text
    except Exception as e:
        awards = np.nan
    return awards

# FUNCTION TO EXTRACT ROLES OF THE CELEBRITIES

In [12]:
def get_role(soup):
    role_list = []
    try:
        li_list = soup.findAll('ul', {'class':'ipc-inline-list ipc-inline-list--show-dividers sc-7f1a92f5-4 kIoyyw baseAlt'})[0].findAll('li')
        for i in li_list:
            role_list.append(i.text)
    except Exception as e:
        role_list.append(np.nan)
    return role_list

In [13]:
get_role(soup)

[nan]

# MAIN LOOP TO ITERATE THROUGH EACH CELEBRITY'S WEBPAGE AND EXTRACT DATA

In [14]:
dob_list = []
height_list = []
role_list = []
award_list = []
birth_list = []
for i in tqdm(link_list, desc="Processing Links"):
    driver.get(f'https://m.imdb.com/{i}')
    #time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    dob_list.append(get_dob(soup))
    height_list.append(get_height(soup))
    birth_list.append(get_birthplace(soup))
    role_list.append(get_role(soup))
    award_list.append(get_awards(soup))

Processing Links:   0%|          | 0/100 [00:00<?, ?it/s]

Processing Links: 100%|██████████| 100/100 [13:12<00:00,  7.93s/it]


# CREATING DATAFRAME FOR COLLECTED DATASET 

In [15]:
df = pd.DataFrame({
    'Rank':np.arange(1,101),
    'Name': name_list,
    'Date_of_birth': dob_list,
    'height': height_list,
    'Role': role_list,
    'Awards': award_list,
    'Famous_for': famous_list,
    'Birth_place': birth_list
})

# CHECKING THE DATASET

In [16]:
df.head()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
0,1,Zawe Ashton,"July 25, 1984",5′ 10″ (1.78 m),"[Actress, Producer, Writer]",1 win & 1 nomination,Velvet Buzzsaw (2019),"Hackney, London, England, UK"
1,2,Brie Larson,"October 1, 1989",5′ 6¾″ (1.70 m),"[Actress, Producer, Director]",66 wins & 68 nominations total,Room (2015),"Sacramento, California, USA"
2,3,Michael Fassbender,"April 2, 1977",6′ (1.83 m),"[Actor, Producer, Executive]",78 wins & 154 nominations total,Shame (2011),"Heidelberg, Baden-Württemberg, West Germany"
3,4,Sydney Sweeney,"September 12, 1997",5′ 3½″ (1.61 m),"[Actress, Producer]",4 wins & 15 nominations total,Euphoria (2019–2025),"Spokane, Washington, USA"
4,5,David Fincher,"August 28, 1962",6′ 0½″ (1.84 m),"[Producer, Director, Writer]",72 wins & 150 nominations total,Se7en (1995),"Denver, Colorado, USA"


In [17]:
df.tail()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
95,96,Emma Dumont,"November 15, 1994",5′ 9″ (1.75 m),"[Actress, Director, Producer]",NaN,Inherent Vice (2014),"Seattle, Washington, USA"
96,97,Mary McDonnell,"April 28, 1952",5′ 8″ (1.73 m),"[Actress, Soundtrack]",4 wins & 15 nominations total,Battlestar Galactica (2004–2009),"Wilkes-Barre, Pennsylvania, USA"
97,98,Tom Hardy,"September 15, 1977",5′ 9″ (1.75 m),"[Actor, Producer, Writer]",22 wins & 86 nominations total,Inception (2010),"Hammersmith, London, England, UK"
98,99,Barry Keoghan,"October 18, 1992",5′ 8″ (1.73 m),[Actor],11 wins & 68 nominations total,The Killing of a Sacred Deer (2017),"Dublin, Ireland"
99,100,Missi Pyle,November 16,5′ 11¾″ (1.82 m),"[Actress, Writer, Producer]",2 wins & 6 nominations,Galaxy Quest (1999),"Houston, Texas, USA"


In [18]:
df.sample(20)

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
68,69,Hailee Steinfeld,"December 11, 1996",5′ 8½″ (1.74 m),"[Actress, Producer, Composer]",26 wins & 61 nominations total,True Grit (2010),"Tarzana, Los Angeles, California, USA"
31,32,Mckenna Grace,"June 25, 2006",5′ 0¾″ (1.54 m),"[Actress, Producer, Additional Crew]",3 wins & 12 nominations total,Gifted (2017),"Grapevine, Texas, USA"
69,70,Elizabeth Lail,"March 25, 1992",5′ 8″ (1.73 m),[Actress],1 nomination,Five Nights at Freddy's (2023),"Williamson County, Texas, USA"
83,84,Anya Taylor-Joy,"April 16, 1996",5′ 8″ (1.73 m),"[Actress, Soundtrack]",19 wins & 44 nominations total,The Queen's Gambit (2020– ),"Miami, Florida, USA"
4,5,David Fincher,"August 28, 1962",6′ 0½″ (1.84 m),"[Producer, Director, Writer]",72 wins & 150 nominations total,Se7en (1995),"Denver, Colorado, USA"
53,54,Kelly Reilly,"July 18, 1977",5′ 6″ (1.68 m),"[Actress, Producer]",7 wins & 12 nominations,Pride & Prejudice (2005),"Surrey, England, UK"
24,25,Millie Bobby Brown,"February 19, 2004",5′ 3½″ (1.61 m),"[Actress, Producer, Executive]",19 wins & 40 nominations total,Enola Holmes (2020),"Málaga, Andalucía, Spain"
5,6,Aria Mia Loberti,NaN,5′ 3½″ (1.61 m),[Actress],NaN,All the Light We Cannot See (2023– ),NaN
58,59,Jeremy Allen White,"February 17, 1991",5′ 7″ (1.70 m),"[Actor, Writer, Producer]",5 wins & 13 nominations total,Shameless (2011–2021),"Brooklyn, New York, USA"
60,61,Shailene Woodley,"November 15, 1991",5′ 8″ (1.73 m),"[Actress, Producer, Additional Crew]",37 wins & 73 nominations total,Divergent (2014),"Simi Valley, California, USA"


# SAVING THE DATASET

In [19]:
df.to_csv('celebrity_data.csv',index= False)